# <center>Cutting stock problem
---

### Importando bibliotecas

In [3]:
import json
import struct
import numpy as np
import pandas as pd
import random
import sys

### Carregando dados

In [4]:
with open("./data/1.json", encoding='utf-8') as arquivo:
    data = json.load(arquivo)

In [5]:
#data

In [6]:
data['Objects']

[{'Length': 52, 'Height': 53, 'Stock': 3, 'Cost': 2756}]

In [7]:
data['Items']

[{'Length': 19, 'Height': 21, 'Demand': 1, 'DemandMax': None, 'Value': 399},
 {'Length': 7, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 140},
 {'Length': 4, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 80},
 {'Length': 15, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 300},
 {'Length': 14, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 280},
 {'Length': 14, 'Height': 19, 'Demand': 1, 'DemandMax': None, 'Value': 266},
 {'Length': 17, 'Height': 19, 'Demand': 1, 'DemandMax': None, 'Value': 323},
 {'Length': 10, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 170},
 {'Length': 13, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 221},
 {'Length': 5, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 85},
 {'Length': 16, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 272},
 {'Length': 20, 'Height': 16, 'Demand': 1, 'DemandMax': None, 'Value': 320},
 {'Length': 5, 'Height': 16, 'Demand': 1, 'DemandMax': None, 'Value': 80},
 {'Len

### Formatando dados

In [140]:
class Container:                     # Classe para os Containers
    def __init__(self, Length, Height, Cost):  
        self.length = Length  
        self.height = Height
        self.cost   = Cost
        self.area   = np.zeros((self.length, self.height))
        self.itens  = []

class Area:
     def __init__(self, Length, Height):  
        self.length  = Length  
        self.height  = Height
        self.area    = self.length * self.height
        self.inicial = [0, 0]
        self.final   = [0, 0]

infoObjs = data['Objects'][0]        # Informações dos Containers de entrada
stock    = infoObjs['Stock']         # Qtd no stock

In [141]:
Lista_Itens = data['Items']          # Lista dos itens de entrada

S = []                               # Conjunto de Container de entrada
for i in range(stock):
    S.append(Container(infoObjs['Length'],
                       infoObjs['Height'], 
                       infoObjs['Cost']))

### Heurística

In [154]:
def Heuristica():
    ctrl = True
    Objeto = S[0]
    E = []
    AM_inicial = Area(Objeto.length, Objeto.height)
    AM_inicial.inicial = [0, 0]
    AM_inicial.final   = [Objeto.length, Objeto.height]
    E.append(AM_inicial)
    cont = 0
    
    for item in Lista_Itens:
        melhorAM = None
        for AM in E:
            if(Acomoda(AM, item)):
                melhorAM = Best(AM, melhorAM, item)
        if(melhorAM is None):
            if(cont < stock):
                cont = cont + 1
                Objeto = S[cont + 1]
                E = []
                AreaObj = Area(Objeto.length, Objeto.height)
                AreaObj.inicial = [0, 0]
                AreaObj.final   = [Objeto.length, Objeto.height]
                E.append(AreaObj)
                Empacotar(item, AreaObj, Objeto)
                Atualizar(E, Objeto)
            else:
                break
        else:
            if(ctrl == True):
                Empacotar(item, melhorAM, Objeto)
                Atualizar(E, Objeto)
                ctrl = False

            
def Acomoda(area, item):
    if(area.length >= item['Length'] and area.height >= item['Height']):
        return True
    
    
def Best(atualAM, melhorAM, item):
    if(melhorAM is None):
        return atualAM
    else:
        if(atualAM.length == item['Length'] and atualAM.height == item['Height']):
            return atualAM
        elif(melhorAM.length == item['Length'] and melhorAM.height == item['Height']):
            return melhorDim
        elif(atualAM.area > melhorDim.area):
            return atualAM
        else:
            return melhorDim
    
    
def Empacotar(item, melhorAM, Objeto):
    Objeto.itens.append(item)
    aleatorio = random.randint(0, 0)
    
    if aleatorio == 0:
        corte = Objeto.area[melhorAM.inicial[0]:melhorAM.final[0], melhorAM.inicial[1]:melhorAM.final[1]]
        for x in range(item['Length']):
            for y in range(item['Height']):
                corte[x][y] = 1    
    elif aleatorio == 1:
        corte = Objeto.area[melhorAM.inicial[0]:melhorAM.final[0], melhorAM.inicial[1]:melhorAM.final[1]]
        for x in range(item['Length']):
            for y in range(item['Height']):
                corte[x][y + (corte.shape[0] - item['Height']) + 1] = 1  
    elif aleatorio == 2:
        corte = Objeto.area[melhorAM.inicial[0]:melhorAM.final[0], melhorAM.inicial[1]:melhorAM.final[1]]
        for x in range(item['Length']):
            for y in range(item['Height']):
                corte[x + (corte.shape[0] - item['Length'])][y] = 1  
    elif aleatorio == 3:
        corte = Objeto.area[melhorAM.inicial[0]:melhorAM.final[0], melhorAM.inicial[1]:melhorAM.final[1]]
        for x in range(item['Length']):
            for y in range(item['Height']):
                corte[x + (corte.shape[0] - item['Length'])][y + (corte.shape[0] - item['Height']) + 1] = 1

                
def Atualizar(E, Objeto):
    aux          = np.empty_like(Objeto.area)
    aux[:]       = Objeto.area
    aux[0][40]   = 1
    aux[26][26]  = 1
    coordInicial = [0, 0]
    coordFinal   = [0, 0]
    indicador = 2
    
    for x in range(Objeto.length):  
        for y in range(Objeto.height):
            if(aux[x][y] == 0):
                corte = aux[x:Objeto.length, y:Objeto.height]
                limX  = corte.shape[0]
                cont = 0
                achou = 0
                
                for k in range(corte.shape[0]):
                    for z in range(corte.shape[1]):
                        if corte[k][z] == 1:
                            achou = 1
                            
                            if k == 0 and cont == 0:
                                print('Condição:1')
                                limX = z
                                area = aux[x:, y:limX-corte.shape[1]]
                                cont = 1
                                break
                                
                            elif limX == corte.shape[0]:
                                print('Condição:2')
                                area = aux[x:corte.shape[0]-k, y:]
                                break
                            
                            elif cont == 1 and z < limX:
                                print('Condição:3')
                                area = aux[x:corte.shape[0]-k, y:limX-corte.shape[1]]
                                break
                        else:
                            continue
                            
                if achou == 0:
                    print('Condição:0')
                    area = corte                    
                
                area[:] = indicador
                
                np.set_printoptions(threshold=np.inf)
                indicador += 1
                print(area)
                
                AreaMaximal = Area(area.shape[0], area.shape[1])

In [ ]:
Heuristica()